# Notebook Used To Generate and Submit Scripts

In [9]:
import os

In [10]:
new_exp_scripts_root_dir = '../../las_sim_tkt_new_exp_scripts_OpenAIGym'
if not os.path.exists(new_exp_scripts_root_dir):
    os.makedirs(new_exp_scripts_root_dir)

In [11]:
tasks=['HalfCheetah-v2', 'Ant-v2', 'Hopper-v2', 'Walker2d-v2']
agents=['TD3', 'SAC', 'PPO']
seeds=['0', '1', '2', '4']

In [13]:
for task in tasks:
    for agent in agents:
        for seed in seeds:
            new_exp_scripts_dir = os.path.join(new_exp_scripts_root_dir,'las_sim_tkt_new_exp_scripts_{}_{}_{}'.format(task, agent, seed))
            if not os.path.exists(new_exp_scripts_dir):
                os.makedirs(new_exp_scripts_dir)
            ######################################################################################################
            exp_job_script_template = './openai_gym_exp_job_tmpl.sh'
            new_exp_job_script = os.path.join(new_exp_scripts_dir, 'exp_job.sh')
            
            sbatch_account = 'def-rgorbet'
            sbatch_gpu = '0'
            sbatch_nodes = '1'
            sbatch_ntasks = '5'
            sbatch_mem_per_cpu = '6144M'
            sbatch_time_d = 0
            sbatch_time_h = 10
            sbatch_time_m = 0

            exp_max_env_setup_time=10  # In minutes
            exp_run_time = sbatch_time_d*24*60 + sbatch_time_h*60 + sbatch_time_m - exp_max_env_setup_time

            las_sim_tkt_dir = '/scratch/lingheng/las_sim_tkt'
            las_sim_tkt_pkg_dir = '/scratch/lingheng/las_sim_tkt_pkg'    # saving packages for offline installation
            las_sim_tkt_dep_dir = '/scratch/lingheng/las_sim_tkt_dep'    # saving installed shared softwares such as Processing, Node, Miniconda, Mujoco
            las_sim_tkt_data_dir = '/scratch/lingheng/las_sim_tkt_data'  # saving experiment data

            nv_driver_version=''  # Adapt deriver version to your system: Graham: 470.103.01, Cedar: 510.47.03
            save_processing_simulator_video='false'

            if (sbatch_gpu == '0' and nv_driver_version != '') or (sbatch_gpu != '0' and nv_driver_version == ''):
                raise Warning("gpu is allocated, but nv_driver_version is not specified!")

            if sbatch_gpu == '0':
                sbatch_gpu=''
            else:
                sbatch_gpu='#SBATCH --gres=gpu:{0}             # Number of GPU(s) per node\n'.format(sbatch_gpu)
                
            # Read exp_job.sh to extract the part that does not need to be changed.
            with open(exp_job_script_template, encoding='utf8') as exp_job_sh_template_file:
                exp_job_sh_content = exp_job_sh_template_file.readlines()
                
            # Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
            with open(new_exp_job_script, 'w', encoding='utf8', newline='\n') as new_exp_job_sh_file:
                # Write the first 30 lines of the new job script, which needs mild modification according to the job requirements
                new_exp_job_sh_file.write('''\
#!/bin/bash
#SBATCH --account={0}
{1}#SBATCH --nodes={2}                # When requesting GPUs per node the number of nodes must be specified.
#SBATCH --ntasks={3}               # number of MPI processes
#SBATCH --mem-per-cpu={4}          # memory; default unit is megabytes
#SBATCH --time={5}-{6}:{7}         # time (DD-HH:MM)

# Define exp_run_time, which will be used by sleep command in exp_run.sh. Make sure to set it to the time 
# deducted by the time needed for all operations before running the experiment in order to neatly kill 
# the processes running different modules. For example, if the time for the job is 12 hours and the estimated 
# all other operations is 10 minutes, then we should set exp_run_time=(12*60-10)m
exp_run_time="{8}m"
exp_run_readable_name='{15}_{16}_{17}'

# Setup related directories
las_sim_tkt_dir={9}
las_sim_tkt_pkg_dir={10}    # saving packages for offline installation
las_sim_tkt_dep_dir={11}    # saving installed shared softwares such as Processing, Node, Miniconda, Mujoco
las_sim_tkt_data_dir={12}  # saving experiment data

# Setup nvidia GPU driver version. To find the driver version, on a compute node, use nvidia-smi to query gpu deriver version:
#   output_nv_driver_version=($(nvidia-smi --query-gpu=driver_version --format=csv))    # Turn return as array with ()
#   nv_driver_version=${{output_nv_driver_version[1]}}    # Retrieve driver version
nv_driver_version={13}             # Graham: 470.103.01, Cedar: 510.47.03
if [ -z "${{nv_driver_version}}" ]; then
    echo "nv_driver_version is unset or set to the empty string"
    add_nvdriver=false
else
    add_nvdriver=true
fi

# Indicate if save processing simulator video (true or false). Set this to true, only when using video is necessary, becuase saving 
#   video is computation expensive.
save_processing_simulator_video={14}

'''.format(sbatch_account, sbatch_gpu, sbatch_nodes, sbatch_ntasks, sbatch_mem_per_cpu, 
           sbatch_time_d, sbatch_time_h, sbatch_time_m, exp_run_time, 
           las_sim_tkt_dir, las_sim_tkt_pkg_dir, las_sim_tkt_dep_dir, las_sim_tkt_data_dir, 
           nv_driver_version, save_processing_simulator_video, task, agent, seed))

                # Write the rest of the exp_job.sh to new job script
                for sh_line in exp_job_sh_content[37:]:
                    new_exp_job_sh_file.write(sh_line)
            ######################################################################################################
            exp_run_script_template = './openai_gym_exp_run_tmpl.sh'
            new_exp_run_script = os.path.join(new_exp_scripts_dir, 'exp_run.sh')
            
            # Read exp_job.sh to extract the part that does not need to be changed.
            with open(exp_run_script_template, encoding='utf8') as exp_run_script_template_file:
                exp_run_sh_content = exp_run_script_template_file.readlines()
            
            resume_learning=''
            new_learning='python $exp_run_code_dir/PL-POMDP/pl/teach.py --env_id {} --rl_reward_type hc_reward --rl_agent {} --seed {}'.format(task, agent, seed)
            
            # Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
            with open(new_exp_run_script, 'w', encoding='utf8', newline='\n') as new_exp_run_sh_file:
                # Write the first part of unchanged lines of the new job script
                for run_line in exp_run_sh_content[:46]:
                    new_exp_run_sh_file.write(run_line)

                # Write the part need to be modified
                new_exp_run_sh_file.write('''\
# Run python script （Note: this is the part need to be changed for different experiment runs.）
if [ -d "$exp_run_data_dir/PL-Teaching-Data" ]; then
  nohup {} &>$exp_run_data_dir/console_python_$(date '+%Y-%m-%d_%H-%M-%S').out &
else
  nohup {} &>$exp_run_data_dir/console_python_$(date '+%Y-%m-%d_%H-%M-%S').out &
fi
'''.format(resume_learning, new_learning))

                # Write the rest lines of the new job script, which needs mild modification according to the job requirements
                for run_line in exp_run_sh_content[52:]:
                    new_exp_run_sh_file.write(run_line)
            ######################################################################################################        
            # Submit 
            print(new_exp_job_script)
            print('bash {} >{}/{}_{}_{}_env_setup_$(date \'+%Y-%m-%d_%H-%M-%S\').out &&'.format(new_exp_job_script,
                                                                                                os.path.join(new_exp_scripts_dir), 
                                                                                                task, agent, seed))
            os.system('bash {} >{}/{}_{}_{}_env_setup_$(date \'+%Y-%m-%d_%H-%M-%S\').out &&'.format(new_exp_job_script,
                                                                                                os.path.join(new_exp_scripts_dir), 
                                                                                                task, agent, seed))

../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_0/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_0/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_0/HalfCheetah-v2_TD3_0_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_1/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_1/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_1/HalfCheetah-v2_TD3_1_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_2/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_2/exp_job.sh >../../las_sim_tkt_new_exp_scripts_Open

../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Ant-v2_PPO_4/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Ant-v2_PPO_4/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Ant-v2_PPO_4/Ant-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_TD3_0/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_TD3_0/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_TD3_0/Hopper-v2_TD3_0_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_TD3_1/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_TD3_1/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_TD3_1/Hopper-v2_TD

In [5]:
job_root_list = os.listdir(new_exp_scripts_root_dir)
for job_root in job_root_list:
    job_list = os.listdir(os.path.join(new_exp_scripts_root_dir, job_root))
    for job_sh in job_list:
        if 'job' in job_sh:
            print(os.path.join(new_exp_scripts_root_dir, job_root, job_sh))
            print('bash {} >{}/{}_{}_{}_env_setup_$(date \'+%Y-%m-%d_%H-%M-%S\').out &&'.format(os.path.join(new_exp_scripts_root_dir, job_root, job_sh),
                                                                                                        os.path.join(new_exp_scripts_root_dir, job_root), task, agent, seed))

../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_SAC_4/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_SAC_4/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_SAC_4/Walker2d-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_0/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_0/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_0/Walker2d-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_PPO_2/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_PPO_2/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp

../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_SAC_1/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_SAC_1/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_SAC_1/Walker2d-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_0/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_0/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_0/Walker2d-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_SAC_2/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_SAC_2/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Wa

../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_4/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_4/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_4/Walker2d-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_2/exp_job.sh
bash ../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_2/exp_job.sh >../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_2/Walker2d-v2_PPO_4_env_setup_$(date '+%Y-%m-%d_%H-%M-%S').out &&


In [6]:
job_root_list = os.listdir(new_exp_scripts_root_dir)
for job_root in job_root_list:
    job_list = os.listdir(os.path.join(new_exp_scripts_root_dir, job_root))
    for job_sh in job_list:
        if 'job' in job_sh:
            print(os.path.join(new_exp_scripts_root_dir, job_root, job_sh))
            os.system('bash {} >{}/{}_{}_{}_env_setup_$(date \'+%Y-%m-%d_%H-%M-%S\').out &&'.format(os.path.join(new_exp_scripts_root_dir, job_root, job_sh),
                                                                                                        os.path.join(new_exp_scripts_root_dir, job_root), task, agent, seed))

../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_SAC_4/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_PPO_0/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Hopper-v2_PPO_2/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Ant-v2_SAC_0/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Ant-v2_TD3_2/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_SAC_0/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_PPO_4/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_HalfCheetah-v2_TD3_2/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Walker2d-v2_TD3_1/exp_job.sh
../../las_sim_tkt_new_exp_scripts_OpenAIGym/las_sim_tkt_new_exp_scripts_Ant-v2_PPO_0/exp_job.s

##

In [43]:
new_exp_scripts_dir = '../../las_sim_tkt_new_exp_scripts'
if not os.path.exists(new_exp_scripts_dir):
    os.makedirs(new_exp_scripts_dir)

## Generate exp_job.sh

In [44]:
exp_job_script_template = './exp_job_tmpl.sh'
new_exp_job_script = os.path.join(new_exp_scripts_dir, 'exp_job.sh')

In [45]:
sbatch_account = 'def-rgorbet'
sbatch_gpu = '0'
sbatch_nodes = '1'
sbatch_ntasks = '5'
sbatch_mem_per_cpu = '6144M'
sbatch_time_d = 0
sbatch_time_h = 10
sbatch_time_m = 0

exp_max_env_setup_time=10  # In minutes
exp_run_time = sbatch_time_d*24*60 + sbatch_time_h*60 + sbatch_time_m - exp_max_env_setup_time

las_sim_tkt_dir = '/scratch/lingheng/las_sim_tkt'
las_sim_tkt_pkg_dir = '/scratch/lingheng/las_sim_tkt_pkg'    # saving packages for offline installation
las_sim_tkt_dep_dir = '/scratch/lingheng/las_sim_tkt_dep'    # saving installed shared softwares such as Processing, Node, Miniconda, Mujoco
las_sim_tkt_data_dir = '/scratch/lingheng/las_sim_tkt_data'  # saving experiment data

nv_driver_version=''  # Adapt deriver version to your system: Graham: 470.103.01, Cedar: 510.47.03
save_processing_simulator_video='false'

if (sbatch_gpu == '0' and nv_driver_version != '') or (sbatch_gpu != '0' and nv_driver_version == ''):
    raise Warning("gpu is allocated, but nv_driver_version is not specified!")

if sbatch_gpu == '0':
    sbatch_gpu=''
else:
    sbatch_gpu='#SBATCH --gres=gpu:{0}             # Number of GPU(s) per node\n'.format(sbatch_gpu)

In [46]:
# Read exp_job.sh to extract the part that does not need to be changed.
with open(exp_job_script_template, encoding='utf8') as exp_job_sh_template_file:
    exp_job_sh_content = exp_job_sh_template_file.readlines()
    

In [47]:
# Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
with open(new_exp_job_script, 'w', encoding='utf8', newline='\n') as new_exp_job_sh_file:
    # Write the first 30 lines of the new job script, which needs mild modification according to the job requirements
    new_exp_job_sh_file.write('''\
#!/bin/bash
#SBATCH --account={0}
{1}#SBATCH --nodes={2}                # When requesting GPUs per node the number of nodes must be specified.
#SBATCH --ntasks={3}               # number of MPI processes
#SBATCH --mem-per-cpu={4}          # memory; default unit is megabytes
#SBATCH --time={5}-{6}:{7}         # time (DD-HH:MM)

# Define exp_run_time, which will be used by sleep command in exp_run.sh. Make sure to set it to the time 
# deducted by the time needed for all operations before running the experiment in order to neatly kill 
# the processes running different modules. For example, if the time for the job is 12 hours and the estimated 
# all other operations is 10 minutes, then we should set exp_run_time=(12*60-10)m
exp_run_time="{8}m"

# Setup related directories
las_sim_tkt_dir={9}
las_sim_tkt_pkg_dir={10}    # saving packages for offline installation
las_sim_tkt_dep_dir={11}    # saving installed shared softwares such as Processing, Node, Miniconda, Mujoco
las_sim_tkt_data_dir={12}  # saving experiment data

# Setup nvidia GPU driver version. To find the driver version, on a compute node, use nvidia-smi to query gpu deriver version:
#   output_nv_driver_version=($(nvidia-smi --query-gpu=driver_version --format=csv))    # Turn return as array with ()
#   nv_driver_version=${{output_nv_driver_version[1]}}    # Retrieve driver version
nv_driver_version={13}             # Graham: 470.103.01, Cedar: 510.47.03
if [ -z "${{nv_driver_version}}" ]; then
    echo "nv_driver_version is unset or set to the empty string"
    add_nvdriver=false
else
    add_nvdriver=true
fi

# Indicate if save processing simulator video (true or false). Set this to true, only when using video is necessary, becuase saving 
#   video is computation expensive.
save_processing_simulator_video={14}

'''.format(sbatch_account, sbatch_gpu, sbatch_nodes, sbatch_ntasks, sbatch_mem_per_cpu, 
           sbatch_time_d, sbatch_time_h, sbatch_time_m, exp_run_time, 
           las_sim_tkt_dir, las_sim_tkt_pkg_dir, las_sim_tkt_dep_dir, las_sim_tkt_data_dir, 
           nv_driver_version, save_processing_simulator_video))
    
    # Write the rest of the exp_job.sh to new job script
    for sh_line in exp_job_sh_content[33:]:
        new_exp_job_sh_file.write(sh_line)

## Generate exp_run.sh

In [32]:
exp_run_script_template = './exp_run_tmpl.sh'
new_exp_run_script = os.path.join(new_exp_scripts_dir, 'exp_run.sh')

In [33]:
# Read exp_job.sh to extract the part that does not need to be changed.
with open(exp_run_script_template, encoding='utf8') as exp_run_script_template_file:
    exp_run_sh_content = exp_run_script_template_file.readlines()
    

In [34]:
resume_exp_dir='2022-05-05_td3/2022-05-05_20-35-52-td3_s0/'
resume_learning='python $exp_run_code_dir/PL-POMDP/pl/teach.py --resume_exp_dir $exp_run_data_dir/PL-Teaching-Data/{}'.format(resume_exp_dir)
new_learning='' # python $exp_run_code_dir/PL-POMDP/pl/teach.py --env_id LAS-Meander --rl_reward_type hc_reward --rl_agent PPO

In [35]:
# Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
with open(new_exp_run_script, 'w', encoding='utf8', newline='\n') as new_exp_run_sh_file:
    # Write the first part of unchanged lines of the new job script
    for run_line in exp_run_sh_content[:68]:
        new_exp_run_sh_file.write(run_line)
    
    # Write the part need to be modified
    new_exp_run_sh_file.write('''\
# Run python script （Note: this is the part need to be changed for different experiment runs.）
if [ -d "$exp_run_data_dir/PL-Teaching-Data" ]; then
  nohup {} &>$exp_run_data_dir/console_python_$(date '+%Y-%m-%d_%H-%M-%S').out &
else
  nohup {} &>$exp_run_data_dir/console_python_$(date '+%Y-%m-%d_%H-%M-%S').out &
fi
'''.format(resume_learning, new_learning))
    
    # Write the rest lines of the new job script, which needs mild modification according to the job requirements
    for run_line in exp_run_sh_content[75:]:
        new_exp_run_sh_file.write(run_line)

## Generate exp_run.sh

In [48]:
exp_run_script_template = './exp_run_tmpl_openai_gym.sh'
new_exp_run_script = os.path.join(new_exp_scripts_dir, 'exp_run.sh')

In [49]:
# Read exp_job.sh to extract the part that does not need to be changed.
with open(exp_run_script_template, encoding='utf8') as exp_run_script_template_file:
    exp_run_sh_content = exp_run_script_template_file.readlines()
    

In [54]:
resume_exp_dir='2022-05-07_td3/2022-05-07_04-23-54-td3_s0/'
resume_learning='python $exp_run_code_dir/PL-POMDP/pl/teach.py --resume_exp_dir $exp_run_data_dir/PL-Teaching-Data/{}'.format(resume_exp_dir)

new_learning='' # python $exp_run_code_dir/PL-POMDP/pl/teach.py --env_id LAS-Meander --rl_reward_type hc_reward --rl_agent PPO

# resume_learning=''
# new_learning='python $exp_run_code_dir/PL-POMDP/pl/teach.py --env_id HalfCheetah-v2 --rl_reward_type hc_reward --rl_agent TD3'

In [55]:
# Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
with open(new_exp_run_script, 'w', encoding='utf8', newline='\n') as new_exp_run_sh_file:
    # Write the first part of unchanged lines of the new job script
    for run_line in exp_run_sh_content[:46]:
        new_exp_run_sh_file.write(run_line)
    
    # Write the part need to be modified
    new_exp_run_sh_file.write('''\
# Run python script （Note: this is the part need to be changed for different experiment runs.）
if [ -d "$exp_run_data_dir/PL-Teaching-Data" ]; then
  nohup {} &>$exp_run_data_dir/console_python_$(date '+%Y-%m-%d_%H-%M-%S').out &
else
  nohup {} &>$exp_run_data_dir/console_python_$(date '+%Y-%m-%d_%H-%M-%S').out &
fi
'''.format(resume_learning, new_learning))
    
    # Write the rest lines of the new job script, which needs mild modification according to the job requirements
    for run_line in exp_run_sh_content[52:]:
        new_exp_run_sh_file.write(run_line)

## Submit Job Scripts

In [ ]:
job_list = os.listdir(new_job_scripts_dir)

In [ ]:
for job_sh in job_list:
    if 'job' in job_sh:
        
        os.system('bash {} '.format(os.path.join(new_job_scripts_dir, job_sh)))

In [ ]:
os.path.join(new_exp_scripts_dir)

## Submit Resumed Job Scripts